In [11]:
# Install packages
!pip install opencv-python

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))) - skipping


In [13]:
# Import librarys
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM

In [14]:
# Create a directory to save the captured frames
output_dir = "captured_frames"
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
else:
    print('Already exist.')

Already exist.


In [15]:
# Create a VideoCapture object to connect to the webcam
# 0 represents the default webcam (you can try different indices if you have multiple webcams)
cap = cv2.VideoCapture(0)

# Create a list to store the captured frames
frames = []

# Capture frames for 5 seconds
capture_time = 20
end_time = cv2.getTickCount() + (cv2.getTickFrequency() * capture_time)

while cv2.getTickCount() < end_time:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Check if the frame was successfully captured
    if not ret:
        print("Failed to capture frame")
        break

    # Save the frame in the output directory
    frame_path = os.path.join(output_dir, f"frame_{len(frames)}.jpg")
    cv2.imwrite(frame_path, frame)

    # Add the frame to the list
    frames.append(frame)

# Release the VideoCapture object
cap.release()

In [ ]:
# Create a grid of subplots to display some of the captured frames
num_frames = len(frames)
if num_frames > 0:
    rows = int(num_frames ** 0.5)
    cols = int(np.ceil(num_frames / rows))

    # Calculate the figure size and subplot dimensions
    figsize = (cols * 3, rows * 3)
    subplot_dims = (rows, cols)

    # Set up the figure and subplots
    fig, axs = plt.subplots(*subplot_dims, figsize=figsize)

    # Display the frames
    for i, frame in enumerate(frames):
        # Convert BGR to RGB for plotting
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Plot the frame
        axs[i // cols, i % cols].imshow(frame_rgb)
        axs[i // cols, i % cols].axis("off")

    # Remove empty subplots
    if num_frames < rows * cols:
        for j in range(num_frames, rows * cols):
            fig.delaxes(axs[j // cols, j % cols])

    # Adjust the layout
    fig.tight_layout()

    # Show the plot
    plt.show()

In [3]:
# Function to extract faces from a dataset of images
def extract_faces_from_dataset(dataset_path):
    face_images = []
    for image_file in os.listdir(dataset_path):
        image_path = os.path.join(dataset_path, image_file)
        image = cv2.imread(image_path)

        # Convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # For each detected face, crop the region of interest (ROI)
        for (x, y, w, h) in faces:
            # Crop the detected face region
            face = gray[y:y + h, x:x + w]

            # Resize the face image to a fixed size
            face = cv2.resize(face, (64, 64))

            # Flatten the face image to a 1D array
            face_images.append(face.flatten())

    return face_images

In [4]:
# Load the face cascade XML file
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [5]:
# Load the dataset of face images
dataset_path = './captured_frames'
face_images = extract_faces_from_dataset(dataset_path)

In [6]:
# Train the One-Class SVM model
model = OneClassSVM()
model.fit(face_images)

OneClassSVM()

In [7]:
# Function to perform face recognition
def recognize_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image using the face cascade classifier
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Crop the detected face region
        face = gray[y:y + h, x:x + w]

        # Resize the face image to a fixed size
        face = cv2.resize(face, (64, 64))

        # Flatten the face image to a 1-D array
        face = face.flatten().reshape(1, -1)

        # Perform face recognition using the SVM model
        result = model.predict(face)

        if result == 1:
            # If the face is recognized as the positive class
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, 'Face is recognized', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            # If the face is recognized as the negative class
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, 'Face is not recognized', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    cv2.imshow('Face Recognition Appliccation for Pooria Azizi', image)

In [9]:
# Open the video capture - 0 is the camera index
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()

    if not ret:
        break

    recognize_faces(frame)

    # Press the (Q) button on keyboard to exit the camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

cap.release()
cv2.destroyAllWindows()